# Forward Error Correction Decoding

In [1]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import xsdfec

ol = Overlay('bitstream/ldpc_loopback.bit')

In [2]:
%store -r llrs pad tx_buffer
initial_data = tx_buffer

In [3]:
def format_llr(v):
    # Perform symmetric saturation
    if v > 7.75:
        v = 7.75
    elif v < -7.75:
        v = -7.75

    v = int(v * pow(2,2)) # Bit shift to expose 2 fractional

    # Convert to binary (2's complement)
    if v >= 0:
        b = '{0:06b}'.format(v)
    else:
        b = '{0:06b}'.format(pow(2,6)+v)

    # Sign extend to 8-bits
    if b[0] == '0': # Number is positive
        b_ex = '00' + b
    else: # Number is negative
        b_ex = '11' + b

    return int(b_ex,2)

In [4]:
llrs_formatted = []
for llr in llrs:
    llrs_formatted.append(format_llr(llr))

In [5]:
fec = ol.ldpc_decoder.sd_fec
ldpc_params = fec.available_ldpc_params()

fec.CORE_ORDER = 0                         # In order outputs
fec.CORE_AXIS_ENABLE = 0                   # Ensure FEC is disabled (000000)

sc_offset = 0
la_offset = 0
qc_offset = 0
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Status','Code ID','SC Offset','LA Offset','QC Offset','Code Name'))
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('------','------','------','------','------','------'))
for code_id in range(len(ldpc_params)):
    
    code_name = ldpc_params[code_id]
    sc_table = fec._code_params.ldpc[code_name]['sc_table']
    la_table = fec._code_params.ldpc[code_name]['la_table']
    qc_table = fec._code_params.ldpc[code_name]['qc_table']
    if type(sc_table) != list : sc_table = [sc_table]

    if (sc_offset + len(sc_table)) > len(fec.LDPC_SC_TABLE):
        print("\nNot enough space for SC table: %s. Terminating.\n" % (code_name))
        break
    if (la_offset*4 + len(la_table)) > len(fec.LDPC_LA_TABLE):
        print("\nNot enough space for LA table: %s. Terminating.\n" % (code_name))
        break
    if (qc_offset*4 + len(qc_table)) > len(fec.LDPC_QC_TABLE):
        print("\nNot enough space for QC table: %s. Terminating.\n" % (code_name))
        break
    
    print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Loaded',code_id, sc_offset, la_offset, qc_offset, code_name))
    fec.add_ldpc_params(code_id, sc_offset, la_offset, qc_offset, code_name)

    sc_offset += len(sc_table)
    la_offset += int(np.ceil(len(la_table)/4))
    qc_offset += int(np.ceil(len(qc_table)/4))
    
fec.CORE_AXIS_ENABLE = 63                  # Enable FEC (111111)

Status     Code ID    SC Offset  LA Offset  QC Offset  Code Name 
------     ------     ------     ------     ------     ------    
Loaded     0          0          0          0          docsis_short
Loaded     1          2          2          21         docsis_medium
Loaded     2          4          4          54         docsis_long
Loaded     3          6          6          97         docsis_init_ranging
Loaded     4          8          8          112        docsis_fine_ranging
Loaded     5          9          9          120        wifi802_11_cr1_2_648
Loaded     6          12         12         163        wifi802_11_cr1_2_1296
Loaded     7          15         15         191        wifi802_11_cr1_2_1944
Loaded     8          18         18         213        wifi802_11_cr2_3_648
Loaded     9          20         20         246        wifi802_11_cr2_3_1296
Loaded     10         22         22         272        wifi802_11_cr2_3_1944
Loaded     11         24         24         294       

In [6]:
code_name = 'docsis_short'
code_id = ldpc_params.index(code_name)

n = fec._code_params.ldpc[code_name]['n']
k = fec._code_params.ldpc[code_name]['k']
p = fec._code_params.ldpc[code_name]['p']

print('Block Length (bits): %s\nInformation Bits: %s\nSub-Matrix Size: %s' % (n, k, p))

Block Length (bits): 1120
Information Bits: 840
Sub-Matrix Size: 56


In [7]:
axis_width = 32 // 8
num_words = 1
pad = axis_width // num_words

K = int(np.ceil(k/8))
N = int(np.ceil(n/8))

tx_buffer = allocate(shape=(n*pad,), dtype=np.uint8)
rx_buffer = allocate(shape=(K*pad,), dtype=np.uint8)

j = 0
for i in range(len(tx_buffer))[::pad]:
    tx_buffer[i] = llrs_formatted[j]
    j += 1

In [8]:
def non_5G_ldpc_ctrl_dec(id, 
                         max_iterations, 
                         term_on_no_change, 
                         term_on_pass, 
                         include_parity_op,
                         hard_op, 
                         code):
    id = '{0:08b}'.format(id)                               # (31:24) uint8
    max_iterations = '{0:06b}'.format(max_iterations)       # (23:18) uint6
    term_on_no_change = '{0:01b}'.format(term_on_no_change) # (17:17) bit1
    term_on_pass = '{0:01b}'.format(term_on_pass)           # (16:16) bit1
    include_parity_op = '{0:01b}'.format(include_parity_op) # (15:15) bit1
    hard_op = '{0:01b}'.format(hard_op)                     # (14:14) bit1
    reserved = '{0:07b}'.format(0)                          # (13:7) uint7
    code = '{0:07b}'.format(code)                           # (6:0) uint7
    
    print('\tID: ', int(id,2))
    print('\tMax Iter: ', int(max_iterations,2))
    print('\tTerm No Change: ', int(term_on_no_change,2))
    print('\tTerm Pass: ', int(term_on_pass,2))
    print('\tParity Pass: ', int(include_parity_op,2))
    print('\tHard OP: ', int(hard_op,2))
    print('\tCode: ', int(code,2))
    
    ctrl = id + max_iterations + term_on_no_change + term_on_pass \
    + include_parity_op + hard_op + reserved + code
    print('\tBinary: ', ctrl)
    
    return(int(ctrl,2))

ctrl_buffer = allocate(shape=(1,), dtype=np.uint32)

ctrl_buffer[0] = non_5G_ldpc_ctrl_dec(code_id, 24, 1, 1, 0, 1, 0)

	ID:  0
	Max Iter:  24
	Term No Change:  1
	Term Pass:  1
	Parity Pass:  0
	Hard OP:  1
	Code:  0
	Binary:  00000000011000110100000000000000


In [9]:
status_buffer = allocate(shape=(1,), dtype=np.uint32)

In [10]:
dma_data = ol.ldpc_decoder.axi_dma_data
dma_ctrl = ol.ldpc_decoder.axi_dma_ctrl

dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_buffer)
dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_buffer)

dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

In [11]:
rx_buffer[::4]

PynqBuffer([177,  22, 201, 167, 201, 130, 107,  16, 163,  57, 136, 205,
            228, 159,  22, 143, 122,  28, 184, 217,  74,  71,  70, 194,
            187, 181, 208, 225, 246, 109, 209, 245,  60, 189, 163, 232,
            212, 252, 192, 210, 138, 114,  85, 210, 250, 250,   9, 100,
              5,   4,  41,  44, 182,  81,  49,  72, 207,  52, 123,  87,
              4, 199, 135, 144,  41, 160, 125, 150, 144, 252,  71, 120,
             35, 103, 254, 254,  22, 168, 181, 198, 194, 203, 151,  15,
             85, 119, 113,  19, 173,  62,  12, 253, 236, 170, 193, 222,
             12, 234,  57,  62, 139, 114, 194,  71, 118], dtype=uint8)

In [12]:
initial_data[::4]

PynqBuffer([177,  22, 201, 167, 201, 130, 107,  16, 163,  57, 136, 205,
            228, 159,  22, 143, 122,  28, 184, 217,  74,  71,  70, 194,
            187, 181, 208, 225, 246, 109, 209, 245,  60, 189, 163, 232,
            212, 252, 192, 210, 138, 114,  85, 210, 250, 250,   9, 100,
              5,   4,  41,  44, 182,  81,  49,  72, 207,  52, 123,  87,
              4, 199, 135, 144,  41, 160, 125, 150, 144, 252,  71, 120,
             35, 103, 254, 254,  22, 168, 181, 198, 194, 203, 151,  15,
             85, 119, 113,  19, 173,  62,  12, 253, 236, 170, 193, 222,
             12, 234,  57,  62, 139, 114, 194,  71, 118], dtype=uint8)

In [13]:
for i in range(len(rx_buffer))[::pad]:
    if rx_buffer[i] != initial_data[i]:
        print('Error detected at index: %d. %d != %d' % (i, rx_buffer[i], initial_data[i]))